In [5]:
import os
WORK = os.environ["WORK"]
PROJ_DIR = f'{WORK}/ADA_Project'
os.chdir(PROJ_DIR)

from glob import glob
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint
from datetime import datetime
import subprocess
import random
import PIL
import matplotlib.pyplot as plt
import base64
import requests
import dotenv
from datetime import datetime
from IPython.display import Markdown, display

In [6]:
def generate_latest_fakes_report(PROJ_DIR, verbose=1):
    
    def upload_img(image, token):
        with open(image, "rb") as file:
            url = "https://api.imgbb.com/1/upload"
            parameters = {
                "key": token,
                "image": base64.b64encode(file.read()),
            }
            res = requests.post(url, parameters)
            link = res.json()
            url = link['data']['url']
            return url
        
    dotenv.load_dotenv(f'{PROJ_DIR}/.env')
    token = os.getenv('TOKEN')

    mb_size = lambda x: Path(x).stat().st_size / (1024*1024)
    dir_up = lambda x, y: "/".join(Path(x).parts[y:])
    
    WORK = Path(PROJ_DIR).parent
    TRfolders_ = f'{PROJ_DIR}/training_runs'
    TRfolders = glob(f'{TRfolders_}/*')
    backups_dir = f'{PROJ_DIR}/.tmp_imgs'
    Path(backups_dir).mkdir(exist_ok=True)

    md_content = []
    latest_fakes = []
    
    now = datetime.now()
    date_time = now.strftime('%m/%d/%Y, %H:%M:%S')
    md_content.append('# Latest fakes\n')
    md_content.append(f'## Date and time: {date_time}\n')

    for folder in TRfolders:
        for num in range(-1, -10, -1):
            files = sorted(glob(folder + "/**/*"))
            files = [x for x in files if 'fakes0' in x]
            if files == []:
                continue
            else:
                latest_fake = sorted(files)[-1]
                latest_fakes.append(latest_fake)
                break
    
    if verbose == 1:
        print('=' * 90,
             '\n\nLatest fakes:\n')
        pprint([x.replace(str(TRfolders_), '') for x in latest_fakes])
        print('\n', '=' * 90, '\n')

    for img in latest_fakes:
        image = PIL.Image.open(img)
        compressed_path = f'{backups_dir}/{Path(img).stem}' + '.jpg'
#         output_dim = tuple(x // 2 for x in image.size)
#         compressed = image.resize(output_dim)
        compressed = image
        compressed.save(compressed_path)
        if verbose == 1:
            print(Path(img).name,
                f'compressed from ({mb_size(img):.2f}MB) to ==> '
                  f'({mb_size(compressed_path):.2f}MB)')
        
        url = upload_img(compressed_path, token)
        if verbose == 1:
            print(f'Link ==> {url}\n')
        img_subdir = dir_up(img, -3)
        
        md_content.append(f'### {img_subdir}\n'
              f'![{Path(compressed_path).name}]({url} "{img_subdir}")'
              '\n\n')

    
    Tstamp = datetime.now().strftime('%m_%d_%Y__%H_%M')
    report_path = f'{PROJ_DIR}/latest_fakes_markdown_reports/{Tstamp}.md'
    with open(report_path, 'w') as f:
        f.write(''.join(md_content).replace('<br></br>', '\n'))
    
    if verbose == 1:
        print(f'Generated a report at ==> {report_path}')
        
    return report_path

In [7]:
report_path = generate_latest_fakes_report(PROJ_DIR, verbose=1)


Latest fakes:

['/AFHQ-WILD_training-runs/00004-AFHQ-WILD_custom-auto2-resumecustom/fakes000204.png',
 '/AFHQ_training-runs/00003-AFHQ_custom-auto2-resumecustom/fakes000065.png',
 '/metfaces_training-runs/00005-metfaces_custom-auto2-resumecustom/fakes000000.png',
 '/StanfordDogs_training-runs/00004-StanfordDogs_custom-auto2-resumecustom/fakes000245.png',
 '/cars196_training-runs/00013-cars196_custom-auto2-resumecustom/fakes000245.png',
 '/AFHQ-DOG_training-runs/00004-AFHQ-DOG_custom-auto2-resumecustom/fakes000077.png',
 '/102flowers_training-runs/00023-102flowers_custom-auto2-resumecustom/fakes000245.png',
 '/gen1_pokemon_training-runs/00008-gen1_pokemon_custom-auto2-resumecustom/fakes000073.png']


fakes000204.png compressed from (16.63MB) to ==> (1.83MB)
Link ==> https://i.ibb.co/PxFk3Pf/7ab3f9d9ae41.jpg

fakes000065.png compressed from (15.76MB) to ==> (1.63MB)
Link ==> https://i.ibb.co/khWwY0M/6b3dff275686.jpg

fakes000000.png compressed from (53.64MB) to ==> (4.42MB)
Link ==> htt

In [8]:
display(Markdown(report_path))

# Latest fakes
## Date and time: 03/20/2021, 18:12:40
### AFHQ-WILD_training-runs/00004-AFHQ-WILD_custom-auto2-resumecustom/fakes000204.png
![fakes000204.jpg](https://i.ibb.co/PxFk3Pf/7ab3f9d9ae41.jpg "AFHQ-WILD_training-runs/00004-AFHQ-WILD_custom-auto2-resumecustom/fakes000204.png")

### AFHQ_training-runs/00003-AFHQ_custom-auto2-resumecustom/fakes000065.png
![fakes000065.jpg](https://i.ibb.co/khWwY0M/6b3dff275686.jpg "AFHQ_training-runs/00003-AFHQ_custom-auto2-resumecustom/fakes000065.png")

### metfaces_training-runs/00005-metfaces_custom-auto2-resumecustom/fakes000000.png
![fakes000000.jpg](https://i.ibb.co/M2zNfbq/a3098b37673a.jpg "metfaces_training-runs/00005-metfaces_custom-auto2-resumecustom/fakes000000.png")

### StanfordDogs_training-runs/00004-StanfordDogs_custom-auto2-resumecustom/fakes000245.png
![fakes000245.jpg](https://i.ibb.co/bPN50Jb/7d6173dbbb83.jpg "StanfordDogs_training-runs/00004-StanfordDogs_custom-auto2-resumecustom/fakes000245.png")

### cars196_training-runs/00013-cars196_custom-auto2-resumecustom/fakes000245.png
![fakes000245.jpg](https://i.ibb.co/g76fW5f/9bb075216214.jpg "cars196_training-runs/00013-cars196_custom-auto2-resumecustom/fakes000245.png")

### AFHQ-DOG_training-runs/00004-AFHQ-DOG_custom-auto2-resumecustom/fakes000077.png
![fakes000077.jpg](https://i.ibb.co/KjybLTX/28ae69934dbe.jpg "AFHQ-DOG_training-runs/00004-AFHQ-DOG_custom-auto2-resumecustom/fakes000077.png")

### 102flowers_training-runs/00023-102flowers_custom-auto2-resumecustom/fakes000245.png
![fakes000245.jpg](https://i.ibb.co/HVmrGR1/65de87d91b0d.jpg "102flowers_training-runs/00023-102flowers_custom-auto2-resumecustom/fakes000245.png")

### gen1_pokemon_training-runs/00008-gen1_pokemon_custom-auto2-resumecustom/fakes000073.png
![fakes000073.jpg](https://i.ibb.co/7QvVvSj/0f6d02a84be7.jpg "gen1_pokemon_training-runs/00008-gen1_pokemon_custom-auto2-resumecustom/fakes000073.png")

